<h1 style="color:DodgerBlue">Индивидальный проект</h1>

<h2 style="color:DodgerBlue">Название проекта:</h2>

----

### Вариант задания №16


<h2 style="color:DodgerBlue">Описание проекта:</h2>

----

Создать базовый класс PaymentMethod в C#, который будет представлять 
различные способы оплаты. На основе этого класса разработать 2-3 производных 
класса, демонстрирующих принципы наследования и полиморфизма. В каждом из 
классов должны быть реализованы новые атрибуты и методы, а также 
переопределены некоторые методы базового класса для демонстрации 
полиморфизма.
##### Требования к базовому классу PaymentMethod:
• Атрибуты: ID способа оплаты (PaymentMethodId), Название способа оплаты 
(MethodName), Минимальная сумма (MinAmount).

 • Методы:

o ProcessPayment(decimal amount): метод для обработки платежа 
указанной суммы.

o CheckMinimumAmount(decimal amount): метод для проверки 
минимальной суммы платежа.

o GetPaymentDetails(): метод для получения деталей способа оплаты.
##### Требования к производным классам:
1. ОнлайнОплата (OnlinePayment): Должен содержать дополнительные 
атрибуты, такие как URL платежной системы (PaymentUrl). 
Метод ProcessPayment() должен быть переопределен для включения URL 
платежной системы в процесс оплаты.
2. БанковскийПеревод (BankTransfer): Должен содержать дополнительные 
атрибуты, такие как Банковские данные (BankData). 
Метод CheckMinimumAmount() должен быть переопределен для проверки 
минимальной суммы платежа с учетом банковских комиссий.
3. Наличные (CashPayment) (если требуется третий класс): Должен содержать 
дополнительные атрибуты, такие как Место выдачи наличных 
(CashPickupPoint). Метод GetPaymentDetails() должен быть переопределен 
для отображения места выдачи наличных.


#### Дополнительное задание
Добавьте к сущестующим классам (базовыму и производным 3-4 атрибута и метода) создайте явную реализации интерфейса и управление зависимостями 


<h2 style="color:DodgerBlue">Реализация:</h2>

----

In [ ]:
using System;
using System.Collections.Generic;

public interface IPaymentMethod
{
    void ProcessPayment(decimal amount);
    bool CheckMinimumAmount(decimal amount);
    string GetPaymentDetails();
    decimal CalculateCommission(decimal amount);
}

public interface ICashback
{
    decimal CalculateCashback(decimal amount);
}

public interface IRefund
{
    void ProcessRefund(decimal amount);
}

public abstract class PaymentMethod : IPaymentMethod
{
    public int PaymentMethodId { get; set; }
    public string MethodName { get; set; }
    public decimal MinAmount { get; set; }
    public string Currency { get; set; }  
    public string PaymentCountry { get; set; }
    public DateTime LastPaymentDate { get; set; }
    public string Description { get; set; }

    public PaymentMethod(int paymentMethodId, string methodName, decimal minAmount, string currency, string paymentCountry, string description)
    {
        PaymentMethodId = paymentMethodId;
        MethodName = methodName;
        MinAmount = minAmount;
        Currency = currency;
        PaymentCountry = paymentCountry;
        Description = description;
    }

    public abstract void ProcessPayment(decimal amount);

    public virtual bool CheckMinimumAmount(decimal amount)
    {
        return amount >= MinAmount;
    }

    public virtual string GetPaymentDetails()
    {
        return $"Способ оплаты: {MethodName}, Минимальная сумма: {MinAmount}, Валюта: {Currency}, Страна: {PaymentCountry}, Описание: {Description}";
    }

    public abstract decimal CalculateCommission(decimal amount);
}

public class OnlinePayment : PaymentMethod
{
    public string PaymentUrl { get; set; }
    public string ConfirmationCode { get; private set; } 

    public OnlinePayment(int paymentMethodId, string methodName, decimal minAmount, string currency, string paymentCountry, string paymentUrl, string description)
        : base(paymentMethodId, methodName, minAmount, currency, paymentCountry, description)
    {
        PaymentUrl = paymentUrl;
    }

    public override void ProcessPayment(decimal amount)
    {
        decimal commission = CalculateCommission(amount);
        ConfirmationCode = GenerateConfirmationCode(); 
        Console.WriteLine($"Онлайн-платеж {amount} через {PaymentUrl}, Код подтверждения: {ConfirmationCode}. Комиссия: {commission}");
        LastPaymentDate = DateTime.Now;
    }

    public override decimal CalculateCommission(decimal amount)
    {
        return amount * 0.02m; 
    }

    private string GenerateConfirmationCode()
    {
        return new Random().Next(1000, 9999).ToString();
    }

    public override string GetPaymentDetails()
    {
        return $"{base.GetPaymentDetails()}, URL: {PaymentUrl}";
    }
}

public class BankTransfer : PaymentMethod
{
    public string BankData { get; set; }

    public BankTransfer(int paymentMethodId, string methodName, decimal minAmount, string currency, string paymentCountry, string bankData)
        : base(paymentMethodId, methodName, minAmount, currency, paymentCountry, "Банковский перевод")
    {
        BankData = bankData;
    }

    public override void ProcessPayment(decimal amount)
    {
        decimal commission = CalculateCommission(amount);
        Console.WriteLine($"Банковский перевод на сумму {amount} через {BankData}. Комиссия: {commission} руб.");
        LastPaymentDate = DateTime.Now; 
    }

    public override decimal CalculateCommission(decimal amount)
    {
        return 50m; 
    }

    public override string GetPaymentDetails()
    {
        return $"{base.GetPaymentDetails()}, через: {BankData}, фиксированная комиссия: 50 руб.";
    }
}

public class CashPayment : PaymentMethod, ICashback, IRefund
{
    public string CashPickupPoint { get; set; }
    public decimal CashbackAccrual { get; set; }
    private decimal _paidAmount = 0;

    public CashPayment(int paymentMethodId, string methodName, decimal minAmount, string currency, string paymentCountry, string cashPickupPoint, decimal cashbackAccrual)
        : base(paymentMethodId, methodName, minAmount, currency, paymentCountry, "Оплата наличными")
    {
        CashPickupPoint = cashPickupPoint;
        CashbackAccrual = cashbackAccrual;
    }

    public override void ProcessPayment(decimal amount)
    {
        _paidAmount += amount;

        decimal cashback = ((ICashback)this).CalculateCashback(amount);

        Console.WriteLine($"Оплата наличными {amount}. Место выдачи: {CashPickupPoint}. Кэшбек: {cashback}");
        LastPaymentDate = DateTime.Now;
    }

    public override decimal CalculateCommission(decimal amount)
    {
        return 0; 
    }

    // Явная реализация интерфейса ICashback
    decimal ICashback.CalculateCashback(decimal amount)
    {
        return amount * CashbackAccrual / 100;
    }

    public void ProcessRefund(decimal amount)
    {
        if (amount > _paidAmount)
        {
            Console.WriteLine("Ошибка: сумма возврата превышает сумму платежа.");
        }
        else
        {
            _paidAmount -= amount;
            Console.WriteLine($"Возврат на сумму {amount} выполнен. Остаток: {_paidAmount}.");
        }
    }

    public override string GetPaymentDetails()
    {
        return $"{base.GetPaymentDetails()}, Место выдачи наличных: {CashPickupPoint}, Кэшбек: {CashbackAccrual}%";
    }
}

public class PaymentProcessor<T> where T : IPaymentMethod
{
    private List<T> _paymentMethods = new List<T>();

    public void AddPaymentMethod(T paymentMethod)
    {
        _paymentMethods.Add(paymentMethod);
    }

    public void RemovePaymentMethod(T paymentMethod)
    {
        _paymentMethods.Remove(paymentMethod);
    }


    public void DisplayInfo()
    {
        foreach (var paymentMethod in _paymentMethods)
        {
            Console.WriteLine(paymentMethod.GetPaymentDetails());
        }
    }
}


public class PaymentService
{
    private readonly IPaymentMethod _paymentMethod;

    // Внедрение зависимости через конструктор
    public PaymentService(IPaymentMethod paymentMethod)
    {
        _paymentMethod = paymentMethod;
    }

    public void MakePayment(decimal amount)
    {
        if (_paymentMethod.CheckMinimumAmount(amount))
        {
            _paymentMethod.ProcessPayment(amount);
        }
        else
        {
            Console.WriteLine($"Ошибка: сумма {amount} ниже минимальной для {_paymentMethod.GetPaymentDetails()}");
        }
    }
}


OnlinePayment onlinePayment = new OnlinePayment(1, "Онлайн-оплата", 100, "RUB", "Россия", "https://payment.example.com", "Оплата через интернет");
BankTransfer bankTransfer = new BankTransfer(2, "Банковский перевод", 100, "RUB", "Россия", "Сбербанк");
CashPayment cashPayment = new CashPayment(3, "Оплата наличными", 50, "RUB", "Россия", "Магазин", 2);


PaymentProcessor<IPaymentMethod> processor = new PaymentProcessor<IPaymentMethod>();


processor.AddPaymentMethod(onlinePayment);
processor.AddPaymentMethod(bankTransfer);
processor.AddPaymentMethod(cashPayment);


Console.WriteLine("Доступные способы оплаты:\n");
processor.DisplayInfo();

Console.WriteLine("\nОбработки платежей\n");

PaymentService onlinePaymentService = new PaymentService(onlinePayment);
PaymentService bankTransferService = new PaymentService(bankTransfer);
PaymentService cashPaymentService = new PaymentService(cashPayment);

decimal paymentAmounts = 200; 

Console.WriteLine($"Обработка онлайн-платежа на сумму: {paymentAmounts}");
onlinePaymentService.MakePayment(paymentAmounts);

Console.WriteLine($"\nОбработка банковского перевода на сумму: {paymentAmounts}");
bankTransferService.MakePayment(paymentAmounts);

Console.WriteLine($"\nОбработка наличного платежа на сумму: {paymentAmounts}");
cashPaymentService.MakePayment(paymentAmounts);
Console.WriteLine();


Console.WriteLine("Возвраты\n");
cashPayment.ProcessRefund(10); 
cashPayment.ProcessRefund(200);


Доступные способы оплаты:

Способ оплаты: Онлайн-оплата, Минимальная сумма: 100, Валюта: RUB, Страна: Россия, Описание: Оплата через интернет, URL: https://payment.example.com
Способ оплаты: Банковский перевод, Минимальная сумма: 100, Валюта: RUB, Страна: Россия, Описание: Банковский перевод, через: Сбербанк, фиксированная комиссия: 50 руб.
Способ оплаты: Оплата наличными, Минимальная сумма: 50, Валюта: RUB, Страна: Россия, Описание: Оплата наличными, Место выдачи наличных: Магазин, Кэшбек: 2%

Обработки платежей

Обработка онлайн-платежа на сумму: 200
Онлайн-платеж 200 через https://payment.example.com, Код подтверждения: 1482. Комиссия: 4.00

Обработка банковского перевода на сумму: 200
Банковский перевод на сумму 200 через Сбербанк. Комиссия: 50 руб.

Обработка наличного платежа на сумму: 200
Оплата наличными 200. Место выдачи: Магазин. Кэшбек: 4

Возвраты

Возврат на сумму 10 выполнен. Остаток: 190.
Ошибка: сумма возврата превышает сумму платежа.
